In [16]:
import pandas as pd
import numpy as np

In [27]:
df = pd.read_csv("vectorbt_reports/fixed_master_backtest_stats.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=['Sharpe Ratio'])
df = df.drop(columns=['Start Value', 'End Value'])
#df = df[df["asset"]=="NVDA"]
#display(df[df["asset"]=="NVDA"])

In [18]:
returns = df['Total Return [%]'] / 100
cumulative_return = (1 + returns).prod() - 1
N = len(returns)  # number of periods
annualized_return = (1 + cumulative_return)**(12 / N) - 1
annualized_volatility = returns.std() * np.sqrt(12)
sharpe_annual = annualized_return / annualized_volatility
print(sharpe_annual)

0.8762860980958181


In [9]:
def compute_annualized_sharpe(df):
    results = []

    for asset, group in df.groupby("asset"):
        # Convert returns from % to decimals
        returns = group['Total Return [%]'] / 100
        benchmark_returns = group['Benchmark Return [%]'] / 100
        N = len(returns)

        if N < 2:
            continue  # Skip assets with insufficient data

        # Strategy Sharpe
        cumulative_return = (1 + returns).prod() - 1
        annualized_return = (1 + cumulative_return)**(12 / N) - 1
        annualized_volatility = returns.std() * np.sqrt(12)
        sharpe_strategy = annualized_return / annualized_volatility if annualized_volatility > 0 else np.nan

        # Benchmark Sharpe
        cumulative_bench = (1 + benchmark_returns).prod() - 1
        annualized_bench_return = (1 + cumulative_bench)**(12 / N) - 1
        annualized_bench_vol = benchmark_returns.std() * np.sqrt(12)
        sharpe_benchmark = annualized_bench_return / annualized_bench_vol if annualized_bench_vol > 0 else np.nan

        results.append({
            'asset': asset,
            'strategy_sharpe': sharpe_strategy,
            'benchmark_sharpe': sharpe_benchmark
        })

    return pd.DataFrame(results)

In [25]:
df = pd.read_csv("vectorbt_reports/expanding_master_backtest_stats.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=['Sharpe Ratio'])
df = df.drop(columns=['Start Value', 'End Value'])
sharpe_df = compute_annualized_sharpe(df)
display(sharpe_df.sort_values(by='strategy_sharpe', ascending=False))

,asset,strategy_sharpe,benchmark_sharpe
3,BTC=F,1.045260e+00,0.571205
14,TSLA,9.520957e-01,0.965867
9,GC=F,6.087711e-01,1.921642
1,AMZN,2.759603e-01,0.416647
0,AAPL,2.083239e-01,0.652176
2,BTC-USD,8.145693e-02,0.654541
8,ES=F,-5.025462e-01,0.789796
13,NVDA,-5.743882e-01,-0.231651
12,MSFT,-1.149881e+00,-0.472800
10,GOOGL,-1.357195e+00,-0.506734


In [26]:
for asset_name in df["asset"].unique():
    asset_df = df[df["asset"] == asset_name]
    print(f"Asset: {asset_name}")
    display(asset_df)

Asset: AAPL


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
0,2024-06-03,2024-07-01,20 days,3.444505,11.709530,100.0,40.992854,1.296052,9 days,2,...,NaN,3 days,NaN,NaN,172.225273,3.855377,65.991519,2.562026,10.144767,AAPL
15,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,5.315042,2.459055,100.0,41.272323,2.724344,11 days 00:00:00,2,...,NaN,6 days 00:00:00,NaN,NaN,265.752082,4.095966,49.965011,2.032969,6.173766,AAPL
30,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,5.689258,3.115993,100.0,20.569515,0.874132,2 days 00:00:00,1,...,NaN,15 days 00:00:00,NaN,NaN,568.925811,7.272844,160.885576,3.580612,19.130472,AAPL
45,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,-0.122128,-0.528387,100.0,39.379514,3.138362,6 days 00:00:00,2,...,-3.044542,7 days 00:00:00,3 days 00:00:00,0.959846,-6.106398,-0.005591,-0.702759,0.998617,-0.009154,AAPL
60,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,-2.542828,0.287553,100.0,39.172703,3.706113,14 days 00:00:00,2,...,-2.866859,10 days 00:00:00,2 days 00:00:00,0.112139,-127.141406,-2.168851,-9.383278,0.665679,-2.603200,AAPL
75,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,-0.473425,0.539220,100.0,58.986060,3.304101,15 days 00:00:00,3,...,-1.618389,7 days 00:00:00,1 days 12:00:00,0.852396,-15.780826,-0.652143,-2.291467,0.881596,-0.879820,AAPL
90,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,1.463437,7.693926,100.0,61.122774,3.036350,12 days 00:00:00,3,...,-2.340036,6 days 00:00:00,1 days 00:00:00,1.603959,48.781239,1.685816,9.999371,1.327075,2.384671,AAPL
105,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,-5.098609,-8.858048,100.0,57.148634,7.929631,8 days 00:00:00,3,...,-3.812185,2 days 00:00:00,1 days 00:00:00,0.317776,-169.953638,-3.670510,-8.246921,0.378791,-4.283345,AAPL
120,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,0.254863,4.567998,100.0,39.824945,4.932140,19 days 00:00:00,2,...,-1.004036,9 days 00:00:00,5 days 00:00:00,1.254092,12.743145,0.285966,0.874563,1.056663,0.407073,AAPL
135,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,-5.155505,-9.344320,100.0,38.258823,6.716227,21 days 00:00:00,2,...,-6.131873,7 days 00:00:00,6 days 00:00:00,0.158388,-257.775245,-2.900779,-8.461441,0.583524,-3.326768,AAPL


Asset: AMZN


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
1,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,1.071798,10.575306,100.0,40.399888,2.422672,13 days 00:00:00,2,...,-0.385755,2 days 00:00:00,2 days 00:00:00,3.741122,53.589888,1.224874,8.865384,1.368195,1.971695,AMZN
16,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-5.801000,-5.182556,100.0,56.332500,9.302333,21 days 00:00:00,3,...,-4.709689,4 days 00:00:00,5 days 12:00:00,0.370253,-193.366657,-3.837140,-6.761465,0.520450,-4.637352,AMZN
31,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,6.800572,-4.535242,100.0,20.680758,5.169062,7 days 00:00:00,1,...,NaN,14 days 00:00:00,NaN,NaN,680.057157,3.920521,35.613090,2.090252,8.864149,AMZN
46,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,4.480331,5.305323,100.0,20.448502,3.185180,4 days 00:00:00,1,...,NaN,13 days 00:00:00,NaN,NaN,448.033060,4.145851,38.467456,1.957615,8.532189,AMZN
61,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,0.237912,2.415070,100.0,39.515772,2.782518,20 days 00:00:00,2,...,-2.542953,14 days 00:00:00,3 days 00:00:00,1.093651,11.895578,0.314678,1.445176,1.050104,0.399936,AMZN
76,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,5.333940,7.813239,100.0,41.860445,2.524082,10 days 00:00:00,2,...,-1.218895,9 days 00:00:00,1 days 00:00:00,5.107966,266.696994,3.804655,54.208166,2.018297,9.384312,AMZN
91,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,5.863392,7.629035,100.0,20.586946,5.422448,8 days 00:00:00,1,...,NaN,13 days 00:00:00,NaN,NaN,586.339194,3.464747,33.727508,1.779107,5.227399,AMZN
106,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,3.294888,6.380477,100.0,59.831538,3.784324,8 days 00:00:00,3,...,-0.907198,7 days 00:00:00,2 days 00:00:00,2.825573,109.829591,3.000399,24.566849,1.656782,5.165643,AMZN
121,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,-2.345741,-8.949958,100.0,39.193669,5.522534,15 days 00:00:00,2,...,-2.860664,1 days 00:00:00,9 days 00:00:00,0.179181,-117.287039,-1.535358,-5.894441,0.736382,-1.818573,AMZN
136,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,-12.494574,-10.090975,100.0,92.380747,12.494574,21 days 00:00:00,5,...,-2.631344,NaN,2 days 04:48:00,0.000000,-249.891474,-6.187234,-7.040978,0.300003,-6.713786,AMZN


Asset: BTC-USD


...

,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
12,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,3.341508,10.449319,100.0,41.168268,0.794097,10 days 00:00:00,2,...,-0.794891,7 days 00:00:00,2 days 00:00:00,5.039538,167.075404,5.778113,103.496184,4.149084,25.497976,MSFT
27,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-9.382601,-8.403215,100.0,56.623609,11.393033,18 days 00:00:00,3,...,-3.227865,NaN,6 days 00:00:00,0.000000,-312.753379,-7.176958,-7.065458,0.325869,-7.445847,MSFT
42,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,2.533515,-0.289229,100.0,20.253625,3.342745,8 days 00:00:00,1,...,NaN,15 days 00:00:00,NaN,NaN,253.351543,2.830404,14.581701,1.574870,4.264249,MSFT
57,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,3.130080,2.608231,100.0,20.313341,2.529812,6 days 00:00:00,1,...,NaN,13 days 00:00:00,NaN,NaN,313.008007,3.531494,29.845008,1.728306,7.143743,MSFT
72,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,-2.124044,0.383459,100.0,58.714475,3.596875,21 days 00:00:00,3,...,-1.475878,5 days 00:00:00,3 days 00:00:00,0.276595,-70.801464,-2.786267,-8.331223,0.569948,-3.294290,MSFT
87,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,-1.586140,-2.074319,100.0,58.998148,4.313605,9 days 00:00:00,3,...,-1.404011,3 days 00:00:00,4 days 12:00:00,0.430583,-52.871322,-1.306228,-5.401441,0.800264,-1.729050,MSFT
102,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,1.284537,1.669581,100.0,20.128602,3.852345,7 days 00:00:00,1,...,NaN,12 days 00:00:00,NaN,NaN,128.453696,1.227794,6.808828,1.274364,1.509102,MSFT
117,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,1.731545,2.290331,100.0,20.173348,2.817164,2 days 00:00:00,1,...,NaN,7 days 00:00:00,NaN,NaN,173.154472,1.653882,14.780575,1.529934,3.505425,MSFT
132,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,-9.107554,-8.015001,100.0,75.749815,11.588286,20 days 00:00:00,4,...,-2.349016,NaN,2 days 12:00:00,0.000000,-227.688838,-5.008327,-6.859624,0.307150,-5.387569,MSFT
147,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,-7.301007,-5.236040,100.0,77.172594,7.301007,18 days 00:00:00,4,...,-1.878227,NaN,3 days 00:00:00,0.000000,-182.525167,-7.147821,-9.584183,0.237353,-7.178538,MSFT


Asset: NVDA


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
13,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,8.359551,8.086959,100.0,62.607617,3.636807,8 days 00:00:00,3,...,-0.921830,4 days 12:00:00,1 days 00:00:00,9.652652,278.651706,4.224174,91.519557,2.230899,9.002074,NVDA
28,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-15.948690,-5.856803,100.0,36.762515,19.999379,15 days 00:00:00,2,...,-8.328746,NaN,5 days 12:00:00,0.000000,-797.434488,-5.536378,-4.720188,0.338871,-5.801483,NVDA
43,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,9.274908,2.008209,100.0,20.928439,9.546157,9 days 00:00:00,1,...,NaN,14 days 00:00:00,NaN,NaN,927.490819,2.889537,32.327583,1.585510,4.694877,NVDA
58,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,12.079043,1.700594,100.0,21.209133,4.843039,7 days 00:00:00,1,...,NaN,13 days 00:00:00,NaN,NaN,1207.904328,4.650497,144.815461,2.357930,12.809031,NVDA
73,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,-0.373924,16.312580,100.0,59.751701,7.715789,11 days 00:00:00,3,...,-3.468675,8 days 00:00:00,1 days 12:00:00,0.946708,-12.464139,0.008847,-0.781016,1.001637,0.011881,NVDA
88,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,-0.313401,-4.184073,100.0,39.658806,5.863790,14 days 00:00:00,2,...,-1.550863,13 days 00:00:00,1 days 00:00:00,0.797716,-15.670051,0.039502,-0.865395,1.006362,0.065429,NVDA
103,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,-2.367120,-0.896930,100.0,59.013804,7.124061,16 days 00:00:00,3,...,-3.896562,2 days 12:00:00,5 days 00:00:00,0.392356,-78.904006,-1.294325,-4.971241,0.782137,-1.945418,NVDA
118,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,-21.407262,-13.870104,100.0,54.891934,21.407262,14 days 00:00:00,3,...,-7.587416,NaN,2 days 08:00:00,0.000000,-713.575416,-5.213986,-4.635991,0.235698,-5.347805,NVDA
133,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,4.433661,10.859653,100.0,39.585308,9.621014,7 days 00:00:00,2,...,-4.296998,17 days 00:00:00,1 days 00:00:00,2.032836,221.683060,1.699619,10.954234,1.263353,2.618019,NVDA
148,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,-8.297149,-16.461000,100.0,37.394294,9.952881,10 days 00:00:00,2,...,-8.884871,13 days 00:00:00,1 days 00:00:00,0.065215,-414.857438,-2.017833,-7.505883,0.698316,-2.613908,NVDA


Asset: TSLA


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
14,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,2.207403,19.042492,100.0,58.858964,4.266855,11 days 00:00:00,3,...,-2.108431,3 days 00:00:00,1 days 00:00:00,1.529211,73.580100,1.504383,11.473149,1.555829,3.481683,TSLA
29,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-6.510177,10.583249,100.0,76.380068,14.144975,15 days 00:00:00,4,...,-2.272341,4 days 00:00:00,2 days 08:00:00,0.028854,-162.754434,-2.024429,-4.755711,0.671829,-2.398304,TSLA
44,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,2.815862,-7.739047,100.0,20.281888,7.847003,7 days 00:00:00,1,...,NaN,13 days 00:00:00,NaN,NaN,281.586225,1.134092,7.057188,1.213994,1.817020,TSLA
59,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,5.013829,21.647747,100.0,38.776152,8.720137,12 days 00:00:00,2,...,-8.637486,13 days 00:00:00,1 days 00:00:00,1.581054,250.691439,1.838529,16.536620,1.412022,2.716849,TSLA
74,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,11.650858,-0.806488,100.0,39.526506,11.207842,17 days 00:00:00,2,...,-8.207121,6 days 00:00:00,3 days 00:00:00,2.421023,582.542884,2.340478,46.610933,2.126791,9.780144,TSLA
89,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,16.108702,28.271435,100.0,70.053249,6.010197,6 days 00:00:00,3,...,-4.158484,3 days 00:00:00,1 days 00:00:00,4.200865,536.956743,4.881576,181.640653,3.503671,14.964126,TSLA
104,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,20.412417,25.060841,100.0,46.607085,1.959277,3 days 00:00:00,2,...,-1.961236,8 days 00:00:00,1 days 00:00:00,9.482694,1020.620872,8.647953,1463.061945,6.227275,33.250264,TSLA
119,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,-17.602596,-4.853743,100.0,53.766084,17.520199,17 days 00:00:00,3,...,-6.229343,NaN,3 days 08:00:00,0.000000,-586.753200,-9.576618,-5.595140,0.145435,-9.231045,TSLA
134,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,-10.908939,-26.778297,100.0,37.720699,12.461947,17 days 00:00:00,2,...,-5.616989,NaN,6 days 12:00:00,0.000000,-545.446936,-4.815199,-6.843791,0.426023,-5.328396,TSLA
149,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,12.166763,-9.370702,100.0,58.981638,8.092825,8 days 00:00:00,3,...,-4.083715,4 days 00:00:00,1 days 00:00:00,2.520671,405.558757,3.375177,64.066153,2.512549,8.620690,TSLA


In [34]:
df = pd.read_csv("vectorbt_reports/fixed_master_backtest_stats.csv")
display(df)


,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
0,2024-04-01,2025-04-01,252 days,10000.0,11396.121645,13.961216,31.265073,100.0,331.369390,11.822727,...,-2.496141,14 days 20:34:17.142857143,3 days 10:17:08.571428571,1.663590,96.667141,0.981400,1.762635,1.220038,1.397310,AAPL
1,2024-04-01 00:00:00,2025-04-01 00:00:00,252 days 00:00:00,10000.0,9026.666221,-9.733338,6.188869,100.0,312.034414,18.499176,...,-3.253165,12 days 12:00:00,4 days 14:24:00,0.696841,-60.833361,-0.521561,-0.745142,0.903925,-0.704521,AMZN
2,2024-04-01 00:00:00,2025-04-01 00:00:00,366 days 00:00:00,10000.0,6558.651130,-34.413489,22.190167,100.0,469.475228,43.436569,...,-3.227581,9 days 12:00:00,5 days 17:27:16.363636363,0.466323,-132.359572,-1.075146,-0.790529,0.793493,-1.552882,BTC-USD
3,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,10318.481933,3.184819,21.270247,100.0,464.047276,26.525027,...,-3.238489,11 days 16:00:00,3 days 20:48:00,1.056547,15.165806,0.312587,0.174435,1.058752,0.472422,BTC=F
4,2024-04-01 00:00:00,2025-03-31 00:00:00,260 days 00:00:00,10000.0,10010.650322,0.106503,2.404336,100.0,291.904316,6.337852,...,-0.827517,15 days 20:00:00,3 days 06:00:00,0.817565,-8.726597,0.056187,0.023596,1.010590,0.082161,CHFUSD=X
5,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,8608.535388,-13.914646,-14.944454,100.0,337.598015,25.352105,...,-2.908873,9 days 04:48:00,6 days 11:04:36.923076923,0.484274,-100.492877,-0.636939,-0.766772,0.895422,-0.867980,CL=F
6,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,9841.252543,-1.587475,-0.723666,100.0,291.156256,5.623870,...,-0.752357,19 days 12:00:00,3 days 12:00:00,0.776253,-12.120943,-0.405289,-0.405799,0.926057,-0.530612,DX-Y_NYB
7,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,9541.781969,-4.582180,-0.777941,100.0,303.302410,6.099051,...,-0.888877,24 days 00:00:00,5 days 20:00:00,0.498137,-35.155275,-1.097988,-1.072794,0.829926,-1.391662,DX=F
8,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,9347.169719,-6.528303,7.162079,100.0,415.993565,15.355496,...,-1.403991,19 days 00:00:00,4 days 06:24:00,0.682712,-33.926081,-0.742341,-0.604377,0.879399,-0.931042,ES=F
9,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,11638.718840,16.387188,39.454500,100.0,399.340716,9.226847,...,-1.122266,12 days 21:20:00,3 days 09:36:00,1.752926,45.712516,1.512859,2.652538,1.279161,2.074132,GC=F
